# Regridding the Lat/Lon Airtemp dataset to H3



In [ ]:
import os

# The variable is named ESMFMKFILE, and it must point to the esmf.mk file

esmfmk = "/Users/akmoch/micromamba/envs/xa_dggs/lib/esmf.mk"
os.environ["ESMFMKFILE"] = esmfmk

In [ ]:
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import xagg as xa
import xarray as xr
import xdggs

In [ ]:
ds = xr.tutorial.load_dataset("air_temperature").load()
ds

In [ ]:
ds.air.isel(time=0).plot()

In [ ]:
from cartopy import crs as ccrs

In [ ]:
fig, axis = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.Orthographic(-90, 30)))

ds.air.isel(time=1).plot(
    ax=axis,
    transform=ccrs.PlateCarree(),  # this is important!
    # usual xarray stuff
    cbar_kwargs={"orientation": "horizontal", "shrink": 0.7},
    robust=True,
)
axis.coastlines()  # cartopy function

In [ ]:
from h3 import h3

In [ ]:
resolution = 2

lon, lat = xr.broadcast(ds.lon, ds.lat)
## %time index = h3.unstable.vect.geo_to_h3(lat.data.ravel(), lon.data.ravel(), resolution)
%time index = [h3.geo_to_h3(la, lo, resolution) for (la, lo) in zip(lat.data.ravel(), lon.data.ravel())]
# index.shape = lon.shape

len(np.unique(index)) / lon.size

In [ ]:
# index.shape
len(index)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({"geometry": None, "id": index}, index=index)

In [ ]:
df

In [ ]:
import shapely
from shapely.geometry import Polygon, mapping, shape

In [ ]:
df["geometry"] = df.apply(
    lambda r: Polygon(h3.h3_to_geo_boundary(r["id"], geo_json=True)), axis=1
)

In [ ]:
df

In [ ]:
shapely.geometry.Polygon(df.iloc[0, 0])

In [ ]:
gdf2 = gpd.GeoDataFrame(df, geometry="geometry", crs=4326)
bb = shapely.geometry.box(-160, 15, -30, 70)

In [ ]:
ax = gdf2.plot()
# gpd.GeoDataFrame({'geometry':[bb]}, crs=4326).plot(ax=ax, color="gray")

In [ ]:
ll = h3.polyfill(bb.__geo_interface__, geo_json_conformant=True, res=2)

In [ ]:
df3 = pd.DataFrame(ll).rename(columns={0: "h3"})

In [ ]:
df3["geometry"] = df3.apply(
    lambda r: Polygon(h3.h3_to_geo_boundary(r["h3"], geo_json=True)), axis=1
)

In [ ]:
gdf3x = gpd.GeoDataFrame(df3, crs=4326)

In [ ]:
gdf3x.plot()

In [ ]:
gdf3x_h3 = gdf3x.set_index("h3", drop=True)
gdf3x_h3

In [ ]:
ds = xr.tutorial.load_dataset("air_temperature").load()
ds

In [ ]:
# Calculate overlaps
# weightmap = xa.pixel_overlaps(ds.rename_dims({"ni": "x", "nj": "y"}),gdf)
weightmap = xa.pixel_overlaps(ds, gdf3x)

In [ ]:
weightmap

In [ ]:
aggregated = xa.aggregate(ds, weightmap)

In [ ]:
aggregated.to_dataset()

In [ ]:
df_out = aggregated.to_dataframe()
df_out

In [ ]:
first_entries = df_out.groupby(level="poly_idx").first()

In [ ]:
type(first_entries)

In [ ]:
fe = first_entries.set_index("h3", drop=True)
fe

In [ ]:
air_df = gdf3x.set_index("h3", drop=True).join(fe)

In [ ]:
type(air_df)

In [ ]:
air_df.plot(column="air", cmap="viridis")

In [ ]:
import geoviews as gv
import geoviews.feature as gf
import matplotlib.pyplot as plt
from geoviews import opts
from geoviews import tile_sources as gvts

%matplotlib inline

gv.extension("matplotlib")

gv.output(fig="png", size=300)

In [ ]:
earth_polyview = gv.Polygons(air_df, vdims=["air"]).opts(
    projection=ccrs.Orthographic(-90, 30)
)


# test geoviews orthographic view
img = (
    earth_polyview.opts(
        projection=ccrs.Orthographic(-90, 30),
        global_extent=False,
        edgecolor="None",
        xaxis=None,
        yaxis=None,
        show_grid=True,
        show_frame=True,
        colorbar=True,
        fig_size=300,
        color="air",
        cmap="viridis",
    )
    * gf.coastline
)

In [ ]:
img

In [ ]:
type(img)

In [ ]:
gv.save(img, "xagg_h3_air.png")

In [ ]:
ds.lon.max().compute()